# INSTALL REQUIREMENTS

In [1]:
%%capture
%pip install transformers datasets accelerate torch evaluate bert_score rouge_score bitsandbytes

In [2]:
model_name = "allenai/led-base-16384"
model_alias = model_name.split('/')[-1].strip()
trainer_output_dir = f"/kaggle/working/{model_alias}_output"
trainer_log_dir = f"/kaggle/working/{model_alias}_logs"
savepath = f"/kaggle/working/custom-{model_alias}"

print("Save path:\t",savepath)
print("Log path:\t", trainer_log_dir)
print("Output path:\t",trainer_output_dir)

Save path:	 /kaggle/working/custom-led-base-16384
Log path:	 /kaggle/working/led-base-16384_logs
Output path:	 /kaggle/working/led-base-16384_output


## IMPORT AND PRE-CONFIGURE MODEL

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
import torch

print("CUDA is available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

CUDA is available: True
CUDA device count: 1
CUDA device name: Tesla P100-PCIE-16GB


In [5]:
import evaluate
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)

2025-05-15 14:24:39.637619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747319079.823095      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747319079.880579      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load pre-trained model

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    inputs_text = examples["article"]
    targets = examples["highlights"]
    inputs = tokenizer(inputs_text, padding="max_length", truncation=True, max_length=1024)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=512)
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]
    inputs["labels"] = labels["input_ids"]
    return inputs

## LOAD DATASET

In [8]:
train_df = pd.read_csv("/kaggle/input/dataset-nlp/train.csv").reset_index(drop=True)
val_df = pd.read_csv("/kaggle/input/dataset-nlp/valid.csv").reset_index(drop=True)
test_df = pd.read_csv("/kaggle/input/dataset-nlp/test.csv").reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/21393 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2674 [00:00<?, ? examples/s]

Map:   0%|          | 0/2675 [00:00<?, ? examples/s]

## CONFIGURE TRAINING PARAMETERS

In [9]:
training_args = TrainingArguments(
    fp16=True,
    bf16=False,
    output_dir=trainer_output_dir,
    save_total_limit=2,
    eval_strategy="epoch", # "no"/"epoch" to disable/enable validation
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=trainer_log_dir,
    logging_steps=200,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,  # Using validation dataset for evaluation
    data_collator=data_collator,
)

## START TRAINING

In [10]:
print("\033[36mStarting training...\033[0m")
trainer.train()
print("\033[33mTraining complete!\033[0m")

Starting training...


Epoch,Training Loss,Validation Loss
1,1.822400,1.785714
2,1.508400,1.753033


Training complete!


# Save trained model

In [11]:
model.save_pretrained(savepath)
tokenizer.save_pretrained(savepath)

('/kaggle/working/custom-led-base-16384/tokenizer_config.json',
 '/kaggle/working/custom-led-base-16384/special_tokens_map.json',
 '/kaggle/working/custom-led-base-16384/vocab.json',
 '/kaggle/working/custom-led-base-16384/merges.txt',
 '/kaggle/working/custom-led-base-16384/added_tokens.json',
 '/kaggle/working/custom-led-base-16384/tokenizer.json')

# Clear VRAM

In [12]:
try:
    import gc

    del trainer
    del model  # If you explicitly defined it outside the trainer
    torch.cuda.empty_cache()  # Clears PyTorch's CUDA memory cache
    gc.collect()  # Forces Python garbage collection
except Exception as e:
    print(e)

# Load trained model


In [13]:
quantization_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
model = AutoModelForSeq2SeqLM.from_pretrained(savepath,quantization_config=quantization_conf, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(savepath)

In [14]:
# Function to preprocess dataset correctly
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch]).to("cuda")
    labels = torch.tensor([item["labels"] for item in batch]).to("cuda")
    return {"input_ids": input_ids, "labels": labels}

eval_dataloader = DataLoader(tokenized_test, batch_size=8, collate_fn=collate_fn)

# Run inference

In [15]:
import torch
from tqdm import tqdm

predictions = []
references = []

# Run inference
for batch in tqdm(eval_dataloader):
    with torch.no_grad():
        outputs = model.generate(batch["input_ids"])

        # Decode predicted texts
        pred_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        # Kiểm tra nếu batch["labels"] có tồn tại
        if "labels" in batch and batch["labels"] is not None:
            labels_tensor = batch["labels"]

            # Đưa về CPU nếu đang ở trên GPU
            if labels_tensor.is_cuda:
                labels_tensor = labels_tensor.cpu()

            # Chuyển thành danh sách số nguyên hợp lệ
            labels_list = labels_tensor.tolist()
            labels_list = [[int(token) for token in seq if 0 <= token < tokenizer.vocab_size] for seq in labels_list]

            # Decode reference texts
            ref_texts = tokenizer.batch_decode(labels_list, skip_special_tokens=True)

            references.extend(ref_texts)

        predictions.extend(pred_texts)


100%|██████████| 335/335 [03:59<00:00,  1.40it/s]


# Evaluating

In [16]:
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

rouge_scores = rouge.compute(predictions=predictions, references=references)
bert_scores = bertscore.compute(predictions=predictions, references=references, lang="en")

from IPython.display import clear_output
clear_output()

# Print results
print("ROUGE:", rouge_scores)
print("BERTScore (averaged):")
print("  Precision:", sum(bert_scores["precision"]) / len(bert_scores["precision"]))
print("  Recall:", sum(bert_scores["recall"]) / len(bert_scores["recall"]))
print("  F1:", sum(bert_scores["f1"]) / len(bert_scores["f1"]))

ROUGE: {'rouge1': 0.2575887454140399, 'rouge2': 0.10968195541260772, 'rougeL': 0.2068619711309897, 'rougeLsum': 0.20680446927086962}
BERTScore (averaged):
  Precision: 0.8907268107494462
  Recall: 0.8410915711215723
  F1: 0.8650076194567101
